Binned all the Features columns to make the data in NN a little less complex and quicker to run through the model. The results still haven't had much of a change

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
df = application_df.drop(["EIN", "NAME", "APPLICATION_TYPE", "SPECIAL_CONSIDERATIONS", "STATUS", "ORGANIZATION", "USE_CASE"], axis=1)
df.head(5)

,AFFILIATION,CLASSIFICATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,Independent,C1000,0,5000,1
1,Independent,C2000,1-9999,108590,1
2,CompanySponsored,C3000,0,5000,0
3,CompanySponsored,C2000,10000-24999,6692,1
4,Independent,C1000,100000-499999,142590,1


In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500
application_counts = df['AFFILIATION'].value_counts()
application_types_to_replace = application_counts[application_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    df['AFFILIATION'] = df['AFFILIATION'].replace(app,"Other")

# Check to make sure binning was successful
df['AFFILIATION'].value_counts()


Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 2000
application_counts = df['CLASSIFICATION'].value_counts()
application_types_to_replace = application_counts[application_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(app,"Other")

# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 1000
application_counts = df['INCOME_AMT'].value_counts()
application_types_to_replace = application_counts[application_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    df['INCOME_AMT'] = df['INCOME_AMT'].replace(app,"Other")

# Check to make sure binning was successful
df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
Other             2790
Name: INCOME_AMT, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500
application_counts = df['ASK_AMT'].value_counts()
application_types_to_replace = application_counts[application_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    df['ASK_AMT'] = df['ASK_AMT'].replace(app,"Other")

# Check to make sure binning was successful
df['ASK_AMT'].value_counts()

5000     25398
Other     8901
Name: ASK_AMT, dtype: int64

In [ ]:

categorical_columns = ['CLASSIFICATION', 'AFFILIATION',
                       'INCOME_AMT', 'ASK_AMT']
df_encoded = pd.get_dummies(df, columns=categorical_columns)
df_encoded.head()


,IS_SUCCESSFUL,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_Other,AFFILIATION_CompanySponsored,AFFILIATION_Independent,AFFILIATION_Other,INCOME_AMT_0,INCOME_AMT_100000-499999,INCOME_AMT_25000-99999,INCOME_AMT_Other,ASK_AMT_5000,ASK_AMT_Other
0,1,1,0,0,0,0,1,0,1,0,0,0,1,0
1,1,0,0,1,0,0,1,0,0,0,0,1,0,1
2,0,0,0,0,1,1,0,0,1,0,0,0,1,0
3,1,0,0,1,0,1,0,0,0,0,0,1,0,1
4,1,1,0,0,0,0,1,0,0,1,0,0,0,1


In [ ]:

from keras.src.feature_column import dense_features_v2
# Split our preprocessed data into our features and target arrays
y = df_encoded['IS_SUCCESSFUL']

X = df_encoded.drop(columns = "IS_SUCCESSFUL")


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:

input_features = X_train_scaled.shape[1]
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation='relu', input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation='tanh'))
nn.add(tf.keras.layers.Dense(units=5, activation='relu'))
nn.add(tf.keras.layers.Dense(units=10, activation='tanh'))
nn.add(tf.keras.layers.Dense(units=15, activation='sigmoid'))
nn.add(tf.keras.layers.Dense(units=10, activation='tanh'))
nn.add(tf.keras.layers.Dense(units=5, activation='relu'))
nn.add(tf.keras.layers.Dense(units=5, activation='tanh'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_86 (Dense)            (None, 5)                 70        
                                                                 
 dense_87 (Dense)            (None, 5)                 30        
                                                                 
 dense_88 (Dense)            (None, 5)                 30        
                                                                 
 dense_89 (Dense)            (None, 10)                60        
                                                                 
 dense_90 (Dense)            (None, 15)                165       
                                                                 
 dense_91 (Dense)            (None, 10)                160       
                                                                 
 dense_92 (Dense)            (None, 5)               

In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
nn.fit(X_train_scaled, y_train, epochs=100, batch_size=16, validation_data=(X_test_scaled, y_test))

Epoch 1/100
1715/1715 [==============================] - 8s 4ms/step - loss: 0.6081 - accuracy: 0.6895 - val_loss: 0.6134 - val_accuracy: 0.6816
Epoch 2/100
1715/1715 [==============================] - 5s 3ms/step - loss: 0.6080 - accuracy: 0.6889 - val_loss: 0.6133 - val_accuracy: 0.6832
Epoch 3/100
1715/1715 [==============================] - 5s 3ms/step - loss: 0.6083 - accuracy: 0.6882 - val_loss: 0.6169 - val_accuracy: 0.6738
Epoch 4/100
1715/1715 [==============================] - 6s 3ms/step - loss: 0.6079 - accuracy: 0.6881 - val_loss: 0.6143 - val_accuracy: 0.6832
Epoch 5/100
1715/1715 [==============================] - 5s 3ms/step - loss: 0.6078 - accuracy: 0.6879 - val_loss: 0.6153 - val_accuracy: 0.6838
Epoch 6/100
1715/1715 [==============================] - 6s 4ms/step - loss: 0.6079 - accuracy: 0.6892 - val_loss: 0.6146 - val_accuracy: 0.6834
Epoch 7/100
1715/1715 [==============================] - 5s 3ms/step - loss: 0.6076 - accuracy: 0.6880 - val_loss: 0.6130 - val_ac

In [ ]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.6135 - accuracy: 0.6835 - 343ms/epoch - 2ms/step
Loss: 0.6135271787643433, Accuracy: 0.6835277080535889
